# Upload the BEANS watkins recordings to HF

#### 1. Download the recordings the way BEANS do it on their [GitHub](https://github.com/earthspecies/beans)

First we will navigate into the mounted data_birdset folder to download the temporary files from the Repo their and install wget & unzip as they are not on the university bash.

In [1]:
%cd '../../../../data_birdset/beans'
!pwd
!sudo apt install wget
!sudo apt install unzip

/workspace/data_birdset/beans
/workspace/data_birdset/beans


/workspace/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Then we will run their script to download the metadata and recordings to have the same splits.

In [2]:
# Their script:
import pandas as pd
from sklearn.model_selection import train_test_split
from plumbum import local, FG

local['wget']['https://archive.org/download/watkins_202104/watkins.zip', '-P', 'data'] & FG
local['unzip']['data/watkins.zip', '-d', 'data/watkins/'] & FG

df = pd.read_csv('data/watkins/annotations.csv')
df = df.apply(lambda r: pd.Series({'path': f"data/watkins/{r['path']}", 'label': r['species']}), axis=1)

df = df[df['label'] != 'Weddell_Seal']    # remove Weddell Seal which has only two instances
# split to train:valid:test = 6:2:2
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=42, shuffle=True, stratify=df['label'])
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=42, shuffle=True, stratify=df_valid_test['label'])
df_train_low, _ = train_test_split(df_train, test_size=0.8, random_state=42, shuffle=True, stratify=df_train['label'])

df_train = df_train.sort_index()
df_train_low = df_train_low.sort_index()
df_valid = df_valid.sort_index()
df_test = df_test.sort_index()

df_train.to_csv('data/watkins/annotations.train.csv')
df_train_low.to_csv('data/watkins/annotations.train-low.csv')
df_valid.to_csv('data/watkins/annotations.valid.csv')
df_test.to_csv('data/watkins/annotations.test.csv')

--2024-04-07 23:28:21--  https://archive.org/download/watkins_202104/watkins.zip
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia803402.us.archive.org/24/items/watkins_202104/watkins.zip [following]
--2024-04-07 23:28:22--  https://ia803402.us.archive.org/24/items/watkins_202104/watkins.zip
Resolving ia803402.us.archive.org (ia803402.us.archive.org)... 207.241.232.192
Connecting to ia803402.us.archive.org (ia803402.us.archive.org)|207.241.232.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 882137083 (841M) [application/zip]
Saving to: ‘data/watkins.zip’

     0K .......... .......... .......... .......... ..........  0%  324K 44m22s
    50K .......... .......... .......... .......... ..........  0%  326K 44m13s
   100K .......... .......... .......... .......... ..........  0% 26.2M 29m39s
   150K .......... ..

Archive:  data/watkins.zip
  inflating: data/watkins/annotations.csv  
   creating: data/watkins/Atlantic_Spotted_Dolphin/
  inflating: data/watkins/Atlantic_Spotted_Dolphin/61025001.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/6102604L.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/6102604J.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/61025008.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/6102500N.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/6102600G.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/61026001.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/6102603C.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/6102603L.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/6102500H.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/61026004.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/6102500E.wav  
  inflating: data/watkins/Atlantic_Spotted_Dolphin/61025004.wav  
  inflating: data/w

#### 2. Convert to HF format

In [3]:
from datasets import Dataset, Audio
import pandas as pd

def load_dataset(split_name):
    df = pd.read_csv(f'data/watkins/annotations.{split_name}.csv')
    dataset = Dataset.from_pandas(df)
    dataset = dataset.cast_column('path', Audio())
    return dataset

splits = ['train', 'train-low', 'valid', 'test']
datasets = {split: load_dataset(split) for split in splits}
datasets['train_low'] = datasets.pop('train-low') # Rename split from train-low to train_low as HF does not accept -
for split, dataset in datasets.items():
    print(dataset[0])



{'Unnamed: 0': 0, 'path': {'path': 'data/watkins/Clymene_Dolphin/83006002.wav', 'array': array([ 0.03955078,  0.05615234, -0.00341797, ...,  0.00341797,
        0.02001953,  0.02929688]), 'sampling_rate': 81920}, 'label': 'Clymene_Dolphin'}
{'Unnamed: 0': 2, 'path': {'path': 'data/watkins/Clymene_Dolphin/8300604P.wav', 'array': array([ 0.00146484, -0.00439453, -0.00097656, ..., -0.01855469,
       -0.02099609, -0.01171875]), 'sampling_rate': 41600}, 'label': 'Clymene_Dolphin'}
{'Unnamed: 0': 10, 'path': {'path': 'data/watkins/Clymene_Dolphin/8300604C.wav', 'array': array([ 0.00537109,  0.00537109, -0.00146484, ..., -0.00390625,
       -0.02294922, -0.02783203]), 'sampling_rate': 81920}, 'label': 'Clymene_Dolphin'}
{'Unnamed: 0': 0, 'path': {'path': 'data/watkins/Clymene_Dolphin/83006002.wav', 'array': array([ 0.03955078,  0.05615234, -0.00341797, ...,  0.00341797,
        0.02001953,  0.02929688]), 'sampling_rate': 81920}, 'label': 'Clymene_Dolphin'}


#### 3. Upload the datasets to HF

In [4]:
for split, dataset in datasets.items():
    dataset.push_to_hub('DBD-research-group/beans_watkins', split=split)

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

...

README.md:   0%|          | 0.00/465 [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/568 [00:00<?, ?B/s]

---
# Download the dataset from HF

Download all splits from the hub. Even when specifying a specific split it still downloads everything! Use `streaming=True` and `cache_dir='...'` for shorter loading times.

In [6]:
from datasets import load_dataset, DatasetDict

#dataset = load_dataset(path='DBD-research-group/beans_watkins', split='train_low')
dataset: DatasetDict = load_dataset(name='default', path='DBD-research-group/beans_watkins')

KeyboardInterrupt: 

In [42]:
# print number of samples and number of distinct classes
print(f"Number of samples: {len(dataset['train'])}")
print(f"Number of distinct classes: {len(dataset['train'].unique('label'))}")
dataset

Number of samples: 1200
Number of distinct classes: 10
